    """
    Aim : Predict demand for an item given a price point,city_name and date

    process:
        -- data preparation
            -- build a baseline; Use yesterday's sales as a baseline
            -- feature engineering
                
        -- model training
            -- Set up an xgboost model for training
        -- cross-validation
            -- set up train/test split and iterate over hyper-parameters
        -- model.predict
            -- Build a model.predict function
        -- metric
            -- RMSE/MAE
        -- Documentation
            -- EDA
            -- Feature Engineering
                -- Transformed features
            -- cross-val scores
            -- 

    Returns:
        _type_: _description_
    """

In [342]:
###################
#### imports ####
###################

In [343]:
import pandas as pd
import sys
import random
import numpy as np
from datetime import date,timedelta

In [344]:
###################
#### read data ####
###################

In [345]:
dpath = 'sample_data.csv'
raw_df = pd.read_csv(dpath)
raw_df
# ,parse_dates=['date','delivery_date'],infer_datetime_format=True)

,user_id,offer_id,city_name,level1_category,level2_category,level3_category,state_name,delivery_date,date,order_qty,rnm,gmv,item_id
0,26272127,Hyd13442,Hyderabad,personal care,baby care,baby diapers & wipes,TELANGANA,2023-06-07 17:14:51,2023-06-06,26,-1547.000000,2587.0,1180414
1,717928,Deal51952,Jaipur,personal care,oral care,tooth paste,RAJASTHAN,2023-06-07 08:10:31,2023-06-06,1,11.000000,89.0,1138014
2,14985062,Ncr21471,WEST DELHI,grocery,"sugar, salt & jaggery",salt,DELHI,2023-06-07 08:31:12,2023-06-06,1,1.051100,25.0,1136461
3,17656864,Deal49788,Manoharpura,beverages,energy & soft drinks,carbonated soft drinks,RAJASTHAN,2023-06-07 13:01:16,2023-06-07,2,9.621002,198.0,1140887
4,26425579,Luc14014,Lucknow,beverages,energy & soft drinks,carbonated soft drinks,UTTAR PRADESH,2023-06-07 09:27:27,2023-06-06,4,21.600006,396.0,1151028
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2758984,26656997,Kol4752,Kolkata,snacks & packaged food,biscuits & cookies,marie biscuit,WEST BENGAL,2023-06-13 08:11:11,2023-06-12,1,1.599998,36.0,1165491
2758985,26656997,Kol19138,Kolkata,cleaning & household care,cleaners,toilet cleaners,WEST BENGAL,2023-06-13 08:11:11,2023-06-12,1,8.860001,62.0,1173182
2758986,6566396,kol17433,Kolkata,beverages,energy & soft drinks,carbonated soft drinks,WEST BENGAL,2023-06-13 10:14:45,2023-06-12,6,16.320007,510.0,1157791
2758987,26844599,Luc4070,Lucknow,"appliances, mobiles acc & more",mobiles & it accessories,speakers,UTTAR PRADESH,2023-06-13 07:59:50,2023-06-12,1,-14.000000,198.0,1146798


### Feature List
'user_id' : unique identifier for user

'item_id' : unique identifier for item          

'offer_id': pricing construct for item_id, can be mapped to multiple item_id

'city_name': unique identifier city_name

'level1_category' : level 1 category level 

'level2_category' : level 2 category level 

'level3_category' : level 3 category level 

'state_name'      : state_name

'date'            : order_date

'delivery_date'   : delivery_date

'order_qty'       : quantity of item purchased

'rnm'             : rupee net margin for the given item

'gmv'             : revenue for the given item


## EDA
-- Spend time on this to identify features + patterns

In [346]:
1_00 * raw_df.isnull().sum()/raw_df.shape[0]

user_id            0.000000
offer_id           0.000000
city_name          0.000000
level1_category    0.000036
level2_category    0.000036
level3_category    0.000000
state_name         0.000000
delivery_date      0.000181
date               0.000000
order_qty          0.000000
rnm                0.000000
gmv                0.000000
item_id            0.000000
dtype: float64

In [347]:
raw_df = raw_df.loc[(raw_df['delivery_date'].notnull()) & (raw_df['level1_category'].notnull()),:]

In [348]:
raw_df.groupby('date')[['order_qty']].sum()

,order_qty
date,
2023-06-01,115733
2023-06-02,125334
2023-06-03,126819
2023-06-04,118543
2023-06-05,122337
2023-06-06,121059
2023-06-07,123461
2023-06-08,123997
2023-06-09,124559


## Feature Transformations

output

In [349]:
from datetime import datetime,timedelta,date

In [350]:
print(type(raw_df['date'][1]))

<class 'str'>


In [351]:
raw_df['date']=raw_df['date'].map(lambda x:datetime.strptime(x,'%Y-%m-%d'))
# print(type(raw_df['date'][1]))

/var/folders/57/17tqrwcd4f15w4vd85kb3bfm0000gq/T/ipykernel_1598/984205745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['date']=raw_df['date'].map(lambda x:datetime.strptime(x,'%Y-%m-%d'))


In [352]:
raw_df['delivery_date']=raw_df['delivery_date'].map(lambda x:datetime.strptime(x[:10],'%Y-%m-%d'))

/var/folders/57/17tqrwcd4f15w4vd85kb3bfm0000gq/T/ipykernel_1598/4170133267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['delivery_date']=raw_df['delivery_date'].map(lambda x:datetime.strptime(x[:10],'%Y-%m-%d'))


In [353]:
raw_df.dtypes

user_id                     int64
offer_id                   object
city_name                  object
level1_category            object
level2_category            object
level3_category            object
state_name                 object
delivery_date      datetime64[ns]
date               datetime64[ns]
order_qty                   int64
rnm                       float64
gmv                       float64
item_id                     int64
dtype: object

In [354]:
raw_df['selling_price'] = raw_df['gmv']/raw_df['order_qty']

/var/folders/57/17tqrwcd4f15w4vd85kb3bfm0000gq/T/ipykernel_1598/2262095660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['selling_price'] = raw_df['gmv']/raw_df['order_qty']


In [355]:
output_df = raw_df.groupby(['item_id','city_name','date'])[['order_qty']].sum().reset_index()

In [356]:
#########################
#### custom features ####
#########################

# -- Any feature transformations that you do has to be at 'item_id','city_name','date';

transform_df = raw_df.groupby(['item_id','city_name','date'])[['selling_price']].mean().reset_index()

### Benchmark creation 

In [357]:
raw_df['next_date'] = raw_df['date'] + timedelta(1)

benchmark_df = raw_df.groupby(['item_id','city_name','next_date'])[['order_qty']].sum().reset_index()

benchmark_df = benchmark_df.rename(columns={'order_qty':'order_qty_lag_of_1'})

/var/folders/57/17tqrwcd4f15w4vd85kb3bfm0000gq/T/ipykernel_1598/3323048202.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['next_date'] = raw_df['date'] + timedelta(1)


##### Merging benchmark with transform to use last day's sales as a feature

In [358]:
transform_df = transform_df.merge(benchmark_df,left_on=['item_id','city_name','date'],right_on=['item_id','city_name','next_date'],how='left')

In [359]:
transform_df1 = transform_df.merge(output_df,on=['item_id','city_name','date'],how='left')
transform_df1['order_qty_lag_of_1'] = transform_df1['order_qty_lag_of_1'].fillna(0)
display(transform_df1)

,item_id,city_name,date,selling_price,next_date,order_qty_lag_of_1,order_qty
0,1135035,Ahmedabad,2023-06-01,193.025000,NaT,0.0,40
1,1135035,Ahmedabad,2023-06-02,192.615385,2023-06-02,40.0,34
2,1135035,Ahmedabad,2023-06-03,193.627451,2023-06-03,34.0,20
3,1135035,Ahmedabad,2023-06-04,185.500000,2023-06-04,20.0,2
4,1135035,Ahmedabad,2023-06-08,182.000000,NaT,0.0,1
...,...,...,...,...,...,...,...
471267,1191699,Lucknow,2023-06-29,168.000000,2023-06-29,1.0,1
471268,1191710,Ahmedabad,2023-06-28,33.000000,NaT,0.0,1
471269,1191710,Ahmedabad,2023-06-29,33.000000,2023-06-29,1.0,2
471270,1191710,Ahmedabad,2023-06-30,33.000000,2023-06-30,2.0,3


In [360]:
#New feature is added, "No of days taken for fulfilling the delivery"

raw_df['date'] = pd.to_datetime(raw_df['date'])
raw_df['delivery_date'] = pd.to_datetime(raw_df['delivery_date'])
raw_df['no_of_days_for_delivery'] = (raw_df['delivery_date'] - raw_df['date']).dt.days

no_of_days_df = raw_df.groupby(['item_id','city_name','date'])[['no_of_days_for_delivery']].mean().reset_index()

extracted_col = no_of_days_df['no_of_days_for_delivery']

transform_df1 = transform_df1.join(extracted_col)

/var/folders/57/17tqrwcd4f15w4vd85kb3bfm0000gq/T/ipykernel_1598/3230077308.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['date'] = pd.to_datetime(raw_df['date'])
/var/folders/57/17tqrwcd4f15w4vd85kb3bfm0000gq/T/ipykernel_1598/3230077308.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['delivery_date'] = pd.to_datetime(raw_df['delivery_date'])
/var/folders/57/17tqrwcd4f15w4vd85kb3bfm0000gq/T/ipykernel_1598/3230077308.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [361]:
#New feature is added, "No. of unique users"
no_of_unique_users_df = raw_df.groupby(['item_id','city_name','date'])[['user_id']].nunique().reset_index()
# display(no_of_unique_users_df)

no_of_unique_users_df = no_of_unique_users_df.rename(columns={'user_id':'no_of_unique_users'})

extracted_user_col = no_of_unique_users_df['no_of_unique_users']
transform_df1 = transform_df1.join(extracted_user_col)


#New feature is added, "Average quantity ordered by an user for the particular item"
transform_df1['avg_quantity'] = transform_df1['order_qty']/transform_df1['no_of_unique_users']


#New feature is added, "No. of unique offers"
no_of_unique_offers_df = raw_df.groupby(['item_id','city_name','date'])[['offer_id']].nunique().reset_index()
no_of_unique_offers_df = no_of_unique_offers_df.rename(columns={'offer_id':'no_of_unique_offers'})

extracted_user_col = no_of_unique_offers_df['no_of_unique_offers']
transform_df1 = transform_df1.join(extracted_user_col)

display(transform_df1)

,item_id,city_name,date,selling_price,next_date,order_qty_lag_of_1,order_qty,no_of_days_for_delivery,no_of_unique_users,avg_quantity,no_of_unique_offers
0,1135035,Ahmedabad,2023-06-01,193.025000,NaT,0.0,40,1.200000,30,1.333333,1
1,1135035,Ahmedabad,2023-06-02,192.615385,2023-06-02,40.0,34,1.384615,26,1.307692,1
2,1135035,Ahmedabad,2023-06-03,193.627451,2023-06-03,34.0,20,1.235294,17,1.176471,1
3,1135035,Ahmedabad,2023-06-04,185.500000,2023-06-04,20.0,2,1.000000,2,1.000000,1
4,1135035,Ahmedabad,2023-06-08,182.000000,NaT,0.0,1,1.000000,1,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
471267,1191699,Lucknow,2023-06-29,168.000000,2023-06-29,1.0,1,1.000000,1,1.000000,1
471268,1191710,Ahmedabad,2023-06-28,33.000000,NaT,0.0,1,1.000000,1,1.000000,1
471269,1191710,Ahmedabad,2023-06-29,33.000000,2023-06-29,1.0,2,1.000000,2,1.000000,1
471270,1191710,Ahmedabad,2023-06-30,33.000000,2023-06-30,2.0,3,1.500000,2,1.500000,1


In [362]:
#Features are as following:
# 1. Selling price - Average price at which the particular item was sold
# 2. Order Quantity - Total number of items sold of that particular kind
# 3. No. of days for delivery to complete 
# 4. No. of unique users - How many unique users ordered for the particular item in a particular city on a particular date
# 5. Average quanity ordered for the particular item = Total quntity of an item / Total unique users
# 6. No. of unique offers - How many unique offers on that particular item

features = ['selling_price','order_qty_lag_of_1','no_of_days_for_delivery','no_of_unique_users','avg_quantity','no_of_unique_offers'] 

### training pipeline

##### xgboost

In [363]:
import xgboost as xgb

In [364]:
def xgb_data(features,transform_df1):
    
    """
    Create training/validation data for xgboost
    Convert to xgb format
    """

    train_df = transform_df1.loc[transform_df1['date']<=datetime(2023,6,28),features]
    train_y = transform_df1.loc[transform_df1['date']<=datetime(2023,6,28),'order_qty']

    d_train = xgb.DMatrix(train_df,train_y)

    test_df = transform_df1.loc[transform_df1['date']>datetime(2023,6,28),features]
    test_y = transform_df1.loc[transform_df1['date']>datetime(2023,6,28),'order_qty']

    d_test = xgb.DMatrix(test_df,test_y)
    
    return d_train,d_test

In [365]:
def xgb_training(features,transform_df1):

    """
    define xgboost parameters
    call xgb_data for training/validation data
    train xgboost model
    """

    xgb_params = {'tree_method':'approx','eval_metric':'rmse','eta': 0.1,'max_depth':10,'subsample': 0.8,
                'objective': 'reg:squarederror','silent': 1,'colsample_bytree': 0.9,'seed':0}

    d_train,d_test = xgb_data(features,transform_df1)

    watchlist  = [(d_train,'train'),(d_test,'eval')]

    rfr = xgb.train(xgb_params, d_train, num_boost_round=100, evals = watchlist,verbose_eval=None ,early_stopping_rounds=50)
    
    return rfr

In [366]:
d_train,d_test = xgb_data(features,transform_df1)

In [367]:
rfr = xgb_training(features,transform_df1)

[16:37:40] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "silent" } are not used.



##### Difference between bagging and boosting?
-- How does xgboost accelerates the gradient boosting method?

### Baseline Creation

In [368]:
transform_df1.loc[transform_df1['date']>datetime(2023,6,28),'order_qty_lag_of_1'] = transform_df1.loc[transform_df1['date']>datetime(2023,6,28),'order_qty_lag_of_1'].fillna(0)

In [369]:
# Needs to be created at item_id + city_name + date; 
btest = transform_df1.loc[transform_df1['date']>datetime(2023,6,28),'order_qty_lag_of_1'].values

### Predict and Test Creation

In [370]:
ypred = rfr.predict(d_test).astype(int)

In [371]:
ytest = transform_df1.loc[transform_df1['date']>datetime(2023,6,28),'order_qty'].values

### Evaluation metrics

In [372]:
from sklearn.metrics import mean_absolute_error

In [373]:
mean_absolute_error(ytest,ypred) # model

1.1730487208733908

In [374]:
mean_absolute_error(ytest,btest) # baseline/benchmark

4.4217695942642985

### Post-Hoc Analysis

-- What features are working for you?
#### 1. Selling price - How the demand is varying with repspect to the price of the product
#### 2. Order quantity - If order quantity is more in particular region, it means demand is more for that product and vice versa
#### 3. No. of unique users - If number of unique users are more, demand is more for that product
#### 4. No. of unique offers - How demand is affected based on the number of unique offers
#### 5. Average quantity - What's the average quantity purchased by every user
#### 6. Delivery time - How delivery time affects the demand of the product. Timely delivery results in higher chances of user ordering again.


-- How can we iterate over the model to improve on the results?

### Saving Model

In [375]:
# saving model to disk; Upload this to drive and share the path with me
#rfr.save_model("/Users/dealshare/ds-workspace/presentation-notebook/saved_models/xgb_model.txt")